In [12]:
import pandas as pd
import numpy as np
from py_files.data_manager import get_cleaned_data
from py_files.helper_funcs import set_np_pd_display_params
from config import data_path
import xgboost as xgb

from sklearn.model_selection import train_test_split

set_np_pd_display_params(np, pd)

# trying nyc on XGBoost vanilla

In [19]:
data_path = "/home/jeffx/ACME/senior/vol3/_Homework/data/nyc-taxi-trip-duration/train.csv"
nyc_data = pd.read_csv(data_path)
rel_cols = [
    'pickup_datetime', 'dropoff_datetime', 'passenger_count', 
    'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
    'dropoff_latitude', 'trip_duration']
data = nyc_data[rel_cols].copy()
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime']).astype('int64') // 10**9
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime']).astype('int64') // 10**9
# scale the pickup and dropoff datetimes with min-max scaling
data['pickup_datetime'] = (data['pickup_datetime'] - data['pickup_datetime'].min()) / (data['pickup_datetime'].max() - data['pickup_datetime'].min())
data['dropoff_datetime'] = (data['dropoff_datetime'] - data['dropoff_datetime'].min()) / (data['dropoff_datetime'].max() - data['dropoff_datetime'].min())

X = data.drop('trip_duration', axis=1).copy().values
y = data['trip_duration'].copy().values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_regres = xgb.XGBRegressor()
xgb_regres.fit(X_train, y_train)
y_pred = xgb_regres.predict(X_test)
y_pred = y_pred.astype(int)
y_pred[y_pred < 0] = 0

print(y_pred[y_pred < 0])

print(np.sum(y_pred >= 0), len(y_pred))

from sklearn.metrics import mean_squared_error
print("RMSE: ", np.sqrt(mean_squared_error(y_test, y_pred)))

# compute the Root Mean Squared Logarithmic Error
def rmsle(y, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y_pred))))
print("RMSLE: ", rmsle(y_test, y_pred))

[]
291729 291729
RMSE:  2924.022960643033
RMSLE:  0.5802788877334573


# predict type of exercise

<h1>Question 5.18</h1>

Apply a random forest regressor or classifier to your final project dataset.

(i) Optimize the choice of hyperparameters, including <code>n_estimators</code>, <code>max_depth</code> and <code>max_features</code>.

(ii) Compare the performance of the random forest to the performance of other types of models on this dataset.

(iii) If appropriate for this dataset, identify the three most and least important features. If this is not appropriate to do for this dataset, explain why it is not.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import fetch_california_housing, load_digits
from sklearn.model_selection import train_test_split
from seaborn import load_dataset as sns_load_dataset
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
import json
import xgboost as xgb

data_path = "/home/jeffx/ACME/WearableHealthPredictor/data"

np.set_printoptions(precision=5, suppress=True)
pd.set_option("display.precision", 15)
np.set_printoptions(formatter={'float': lambda x: "{0:0.7f}".format(x)})
pd.set_option('display.float_format', lambda x: "{0:0.7f}".format(x))
pd.set_option('display.max_columns', None)

def movecol(df, cols_to_move=[], ref_col='', place='After'):
    """Moves columns around in a dtaframe
    Examples:
    1)
        df cols ['A', 'B', 'C', 'D', 'X', 'Y', 'Z']
        movecol(df, cols_to_move=['X', 'Y', 'Z'], ref_col='B', place='After')
        df cols ['A', 'B', 'X', 'Y', 'Z', 'C', 'D']]
        
    2)
        df cols ['A', 'B', 'C', 'D', 'X', 'Y', 'Z']
        movecol(df, cols_to_move=['X', 'Y', 'Z'], ref_col='A', place='Before')
        df cols ['X', 'Y', 'Z', 'A', 'B', 'C', 'D']
    """
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    return(df[seg1 + seg2 + seg3])

In [8]:
# load in the csv and select only the relevant columns
df_exercise = pd.read_csv(f"{data_path}/activity_environment_data.csv")
rel_cols = [
    'Steps', 'Calories_Burned', 'Distance_Covered', 
    'Exercise_Duration', 'Exercise_Intensity', 
    'Ambient_Temperature', 'Altitude', 'UV_Exposure'] + ['Exercise_Type']
df_exercise = df_exercise[rel_cols]

# convert exercise intensity to a numeric value
df_exercise['Exercise_Intensity'] = df_exercise['Exercise_Intensity'].fillna('None')
df_exercise['Exercise_Intensity'] = df_exercise['Exercise_Intensity'].map(
    {'None': 0, 'Low':1, 'Moderate':2, 'High':3}
)
display(df_exercise)

for exercise_type, frame in df_exercise.groupby('Exercise_Type'):
    print(exercise_type)
    display(frame.describe())

,Steps,Calories_Burned,Distance_Covered,Exercise_Duration,Exercise_Intensity,Ambient_Temperature,Altitude,UV_Exposure,Age,Weight,Height,Medication,Smoker,Alcohol_Consumption,Sleep_Duration,Deep_Sleep_Duration,REM_Sleep_Duration,Wakeups,Snoring,Heart_Rate,Blood_Oxygen_Level,ECG,Calories_Intake,Water_Intake,Stress_Level,Skin_Temperature,Body_Fat_Percentage,Muscle_Mass,Health_Score,Exercise_Type_None,Exercise_Type_Running,Exercise_Type_Strength Training,Exercise_Type_Yoga,Gender_Female,Gender_Male,Gender_Other,Medical_Conditions_Diabetes,Medical_Conditions_Hypertension,Medical_Conditions_None,Mood_Anxious,Mood_Happy,Mood_Neutral,Mood_Sad
0,9146,457.3000000,7.3168000,1.2196752,1,27.2715255,1739.6906621,4.0691325,61,88.6776227,178.5242311,0,0,1,6.5182059,2.7850332,3.7331726,4,1,147,90.6585051,0,2444.9858078,3.1055560,2,32.7529927,17.4329853,77.5994483,25.9920710,0,1,0,0,0,0,1,1,0,0,0,0,1,0
1,9362,468.1000000,7.4896000,1.3783987,2,24.4302885,4102.0428236,1.7236723,24,86.7420841,200.3463580,1,0,1,7.5224464,0.8743905,6.6480559,2,1,147,97.1333603,1,2199.7770200,2.4411184,3,35.0100441,23.9577925,71.7189357,64.3084497,0,0,0,1,0,1,0,0,0,1,1,0,0,0
2,21,1.0500000,0.0168000,0.8038566,2,29.3405739,1794.8999355,0.1356468,81,61.6560536,195.9713561,1,0,1,8.5833860,6.7604764,1.8229096,2,0,176,99.5555940,0,2182.7556405,1.2403405,1,36.7975232,22.9955332,59.8749357,78.0035458,0,0,1,0,0,0,1,0,1,0,0,0,0,1
3,13252,662.6000000,10.6016000,0.0000000,0,25.2892301,4025.8409215,4.4083426,70,141.5137936,164.1782164,0,0,2,4.4257670,2.6455959,1.7801711,2,1,157,91.4448998,1,1451.2188884,1.6823830,2,32.6145921,13.4106586,67.5280981,16.4541064,1,0,0,0,0,0,1,0,1,0,0,1,0,0
4,21,1.0500000,0.0168000,0.8073890,3,21.8535900,3422.3814469,2.8207389,30,80.5528723,146.9206749,1,1,2,6.0089185,4.4406717,1.5682468,3,0,86,96.0921028,1,1583.5627734,1.4618618,3,33.6023194,24.9200196,54.7414867,42.5282021,0,0,0,1,0,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,12443,622.1500000,9.9544000,0.0000000,0,34.4287708,4736.9699928,7.1570933,16,143.3017415,152.8226165,0,0,2,8.3280844,0.9170085,7.4110760,0,0,75,92.1295817,0,2482.9329114,1.0350767,3,34.7457207,33.7413221,51.0810337,30.9890371,1,0,0,0,1,0,0,0,1,0,0,0,1,0
9996,4492,224.6000000,3.5936000,0.5922883,3,32.0682396,3763.9105488,6.6371734,46,89.4470035,168.0858938,0,1,0,5.3550883,4.8349214,0.5201669,3,0,62,93.0392477,1,2196.0323783,1.0387019,1,37.6687511,16.1439119,45.3212715,33.4796013,0,0,0,1,0,0,1,0,1,0,1,0,0,0
9997,4081,204.0500000,3.2648000,0.8884620,2,38.7766199,2726.0245117,0.9435912,62,43.2842742,200.5363945,0,0,0,9.0167520,2.6278392,6.3889129,2,1,129,95.9278843,0,1259.9328589,2.0630284,2,36.3538274,24.0031213,69.1090824,63.3615507,0,0,0,1,1,0,0,0,0,1,0,0,0,1
9998,10319,515.9500000,8.2552000,0.9225235,1,34.3193926,2391.1961970,1.5939005,27,86.4364945,162.1254260,0,0,1,4.4651591,0.7885863,3.6765728,0,1,93,100.4003758,0,2156.6865429,1.6190091,2,37.4438237,10.1343273,36.2259620,60.7952665,0,0,1,0,1,0,0,1,0,0,0,1,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 40 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Steps                            10000 non-null  int64  
 1   Calories_Burned                  10000 non-null  float64
 2   Distance_Covered                 10000 non-null  float64
 3   Exercise_Duration                10000 non-null  float64
 4   Exercise_Intensity               10000 non-null  int64  
 5   Ambient_Temperature              10000 non-null  float64
 6   Altitude                         10000 non-null  float64
 7   UV_Exposure                      10000 non-null  float64
 8   Age                              10000 non-null  int64  
 9   Weight                           10000 non-null  float64
 10  Height                           10000 non-null  float64
 11  Medication                       10000 non-null  int64  
 12  Smoker             

None

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load your data (replace 'your_data.csv' with the actual path to your data)
data = df_exercise

# Select the features you want to use for clustering
features = data[['Steps', 'Calories_Burned', 'Distance_Covered', 'Exercise_Duration',
                 'Exercise_Intensity', 'Ambient_Temperature', 'Altitude', 'UV_Exposure']].copy().values

# Standardize the data (important for K-Means)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Create a K-Means model with 3 clusters
kmeans = KMeans(n_clusters=3)

# Fit the model to your data
kmeans.fit(scaled_features)

# Add cluster labels to your DataFrame
data['Cluster'] = kmeans.labels_

# View the results, e.g., the count of data points in each cluster
cluster_counts = data['Cluster'].value_counts()
print(cluster_counts)

pca = PCA(n_components=2)
pca_data = pca.fit_transform(scaled_features)
print(pca_data.shape)

# You can also plot the clusters if you want to visualize the results
plt.scatter(pca_data[:,0], pca_data[:,1], c=kmeans.labels_, cmap='viridis', alpha=0.1)
plt.show()


In [ ]:
# load in each of the dataframes
data_path = "/home/jeffx/ACME/WearableHealthPredictor/data"
df = pd.read_csv(f"{data_path}/health_data_clean.csv")

# merge the Exercise_Type columns together to get it as a predicted column
all_exercise_type_cols = ["Exercise_Type_None",	"Exercise_Type_Running", "Exercise_Type_Strength Training", "Exercise_Type_Yoga"]
exercise_type_cols = ["Exercise_Type_Running", "Exercise_Type_Strength Training", "Exercise_Type_Yoga"]
for i, col in enumerate(exercise_type_cols):
    df[col] = df[col] * (i + 1)
df["Exercise_Type"] = df[exercise_type_cols].sum(axis=1) - 1
df = df.drop(columns=all_exercise_type_cols)
df = df.loc[df['Exercise_Type'] != -1, :]

# only consider the relevent columns
rel_cols = [
    'Steps', 'Calories_Burned', 'Distance_Covered', 
    'Exercise_Duration', 'Exercise_Intensity', 
    'Ambient_Temperature', 'Altitude', 'UV_Exposure']

df = df[rel_cols + ["Exercise_Type"]]

# split the data into X and y for train and test
X_df = df.drop(columns=["Exercise_Type"])
features = X_df.columns.tolist()
X = X_df.copy().values

y = df["Exercise_Type"].copy().values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:
display(df)

for exercise_type, frame in df.groupby("Exercise_Type"):
    print(exercise_type)
    display(frame.describe())

### train optimal RandomForestClassifier

In [ ]:
# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 3, 5, 10, 20],
    'max_features': [None, 'sqrt', 'log2']
}

# Create a GridSearchCV object and get the optimal parameters and best score
rf = RandomForestClassifier(warm_start=False)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1).fit(X_train, y_train)

# get the best params and accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

# save the best_params and best_accuracy so I don't have to rerun the GridSearchCV again
import json
with open("best_params.json", "w") as f:
    obj = {
        'best_accuracy': float(round(best_accuracy, 5)), 
        'best_params':best_params
    }
    json.dump(obj, f)

In [ ]:
# load the best params and accuracy
with open("best_params.json") as file:
    obj = json.load(file)
    best_accuracy = obj['best_accuracy']
    best_params = obj['best_params']

# train a new model using the optimal parameters
rf_optimal = RandomForestClassifier(**best_params, warm_start=False)
rf_optimal.fit(X_train, y_train)
rf_predictions = rf_optimal.predict(X_test)
rf_accuracy = rf_optimal.score(X_test, y_test)

# select the top 5 features
feature_importance = rf_optimal.feature_importances_
features_sorted = sorted(list(zip(features, feature_importance)), key=lambda x: x[1], reverse=True)
top_5_features = features_sorted[:5]
top_features, top_feature_scores = map(list, zip(*top_5_features))

# print out the best parameters and the Accuracy
print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)
print("RandomForest Accuracy:", rf_accuracy)
print("Top 5 Features:", top_features)
print("Top 5 Feature Scores:", top_feature_scores)

### train an XGBoost Classifier

In [ ]:
# train and predict with a default XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

# get the accuracy and feature importance
xgb_accuracy = xgb_model.score(X_test, y_test)
xgb_feature_importance = xgb_model.feature_importances_
xgb_features_sorted = sorted(list(zip(features, xgb_feature_importance)), key=lambda x: x[1], reverse=True)
xgb_top_5_features = xgb_features_sorted[:5]
xgb_top_features, xgb_top_feature_scores = map(list, zip(*xgb_top_5_features))

# print out the accuracy and top 5 features
print("XGB Accuracy:", xgb_accuracy)
print("XGB Top 5 Features:", xgb_top_features)
print("XGB Top 5 Feature Scores:", xgb_top_feature_scores)

### train a decision tree classifier

In [ ]:
# train and predict with a default DecisionTree model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)

# get the accuracy and feature importance
dt_accuracy = dt_model.score(X_test, y_test)
dt_feature_importance = dt_model.feature_importances_
dt_features_sorted = sorted(list(zip(features, dt_feature_importance)), key=lambda x: x[1], reverse=True)
dt_top_5_features = dt_features_sorted[:5]
dt_top_features, dt_top_feature_scores = map(list, zip(*dt_top_5_features))

# print out the accuracy and top 5 features
print("DT Accuracy:", dt_accuracy)
print("DT Top 5 Features:", dt_top_features)
print("DT Top 5 Feature Scores:", dt_top_feature_scores)

### analysis

In [ ]:
# display an accuracy table for the 3 models
accuracy_df = pd.DataFrame({
    'Model': ['RandomForest', 'XGBoost', 'DecisionTree'],
    'Accuracy': [rf_accuracy, xgb_accuracy, dt_accuracy],
    'Optimized': [True, False, False]
})
display(accuracy_df)

In [ ]:
# plot each of the ROC curves
def plot_roc_auc(fpr, tpr, roc_auc, classifier, data_name):
    """takes in the false-positive-rate, the true-positive-rate and the
    area-under-the-curve, the name of the classifier, and the name of the dataset
    and plots the ROC curver with the AUC
    """
    plt.plot(fpr, tpr, color='yellow', lw=5, label=f"{classifier} for {data_name}")
    plt.plot([0, 1], [0, 1], color='black', linestyle='--')
    plt.xlim([0.0, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve, AUC={round(roc_auc, 3)}')
    plt.legend(loc='lower right')

# for each model plot the ROC curve for each class
models = [('RandomForest', rf_optimal), ('XGBoost', xgb_model), ('DecisionTree', dt_model)]
classes = ["Running", "Strength Training", "Yoga"]

for i, (model_name, model) in enumerate(models):
    
    probs = model.predict_proba(X_test)
    print(probs)
    
    # for each class
    for j, probs_col in enumerate(probs.T):
        curr_class = classes[j]
        
        # select only the rows that are the current class
        mask = y_test == j
        y_test_curr_class = np.zeros(len(y_test))
        y_test_curr_class[mask] = 1
        
        # get the ROC curve and the AUC (Area Under the Curve) values
        fpr, tpr, thresholds = roc_curve(y_test_curr_class, probs_col)
        roc_auc = auc(fpr, tpr)
        plt.subplot(3, 4, i*4 + j+1)
        plot_roc_auc(fpr, tpr, roc_auc, model_name, curr_class)
    
# show the final plot
plt.gcf().set_size_inches(20,15)
plt.show()